<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-12 13:02:36
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 58.47 K (0.44%)
Current PnL: -21.91 L (-14.9%)
CY Booked + Current PnL: -13.64 L (-9.27%)
-------------------
Total profit:  2.26 L
Total loss:  -24.17 L
-------------------
Total Booked + Current PnL: 13.34 L (11.12%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.22%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.38 L (61.2%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,3.93,218426.0,16946.0,10266.0,0.10,8.41,4.70,13.51,61.0,1.65,1.67,22.07,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.48,H-MC,13.95,160590.0,34390.0,11787.0,-0.22,27.25,7.34,36.59,111.0,2.92,1.23,43.18,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.72,106672.0,2360.0,14294.0,-0.91,2.26,13.40,15.96,11.0,0.17,0.82,19.54,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,93.17,M-SC,10.73,83192.0,-17585.0,17678.0,-0.17,-17.45,21.25,0.09,243.0,-0.99,0.64,9.30,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-26.40,H-SC,9.12,219715.0,26695.0,21334.0,0.38,13.83,9.71,24.89,134.0,1.25,1.68,40.41,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.00,-9.60,H-MC,4.01,196618.0,-410.0,90031.0,-1.16,-0.21,45.79,45.49,103.0,-0.00,1.51,12.81,XY24,BTT,FMCG
56,NATIONALUM,189.63,247.44,-50.29,H-MC,2.10,99937.0,-567.0,31210.0,0.80,-0.56,31.23,30.49,80.0,-0.02,0.77,31.66,MH,ATH,METALS
92,WIPRO,243.46,420.00,-14.25,M-LC,6.84,150598.0,-347.0,109801.0,0.53,-0.23,72.91,72.51,54.0,-0.00,1.15,5.78,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.18,H-LC,2.01,207692.0,3192.0,38215.0,0.74,1.56,18.40,20.25,10.0,0.08,1.59,10.43,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.19,H-LC,2.41,169259.0,-32127.0,119734.0,0.55,-15.95,70.74,43.51,27.0,-0.27,1.30,17.82,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.63,184086.0,-16224.0,76193.0,0.93,-8.10,41.39,29.94,17.0,-0.21,1.41,27.18,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-16.37,H-LC,3.78,128642.0,-136784.0,193696.0,-0.50,-51.53,150.57,21.44,8.0,-0.71,0.99,3.81,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-19.84,H-LC,5.42,160072.0,3895.0,22202.0,0.23,2.49,13.87,16.71,19.0,0.18,1.23,20.21,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SAMMAANCAP,170.35,326.0,-164.77,M-SC,4.46,74586.0,-27624.0,121016.0,8.12,-27.03,162.25,91.37,206.0,-0.23,0.57,21.08,XY25,NTT,FINANCE
29,EASEMYTRIP,18.12,26.4,-6.39,M-SC,23.46,90164.0,-86651.0,167444.0,4.88,-49.01,185.71,45.70,197.0,-0.52,0.69,4.88,XY24,NTT,TRAVEL
6,ASIANTILES,75.41,137.0,6733.33,L-SC,6.82,75001.0,-18809.0,95431.0,3.81,-20.05,127.24,81.67,270.0,-0.20,0.57,45.24,XR,NTT,CERAMICS
79,TATAMOTORS,761.77,1065.0,-24.80,M-LC,5.51,237311.0,-39212.0,149292.0,3.16,-14.18,62.91,39.81,55.0,-0.26,1.82,13.73,XY24,NTT,AUTO
43,INDIGOPNTS,1407.73,1408.0,109.82,M-SC,3.39,137628.0,-36931.0,36967.0,2.88,-21.16,26.86,0.02,221.0,-1.00,1.05,19.34,OX40N,NTT,PAINTS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.00,-35.58,M-SC,6.10,96330.0,-32340.0,77642.0,-1.93,-25.13,80.60,35.20,209.0,-0.42,0.74,0.00,X40,NTT,FOOTWEAR
15,BLUESTARCO,1646.70,2326.38,0.44,H-MC,7.45,173390.0,8720.0,59247.0,-1.59,5.30,34.17,41.28,91.0,0.15,1.33,13.70,X40N,ATH,AC
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.89,130612.0,5315.0,53695.0,-1.36,4.24,41.11,47.09,141.0,0.10,1.00,63.47,MH,ATH,POWER
25,DABUR,505.20,735.00,-9.60,H-MC,4.01,196618.0,-410.0,90031.0,-1.16,-0.21,45.79,45.49,103.0,-0.00,1.51,12.81,XY24,BTT,FMCG
37,HINDUNILVR,2413.81,2723.00,-11.32,H-LC,5.89,279070.0,8723.0,25898.0,-1.06,3.23,9.28,12.81,28.0,0.34,2.14,16.71,XY25,NTT,FMCG


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.17,M-SC,10.73,83192.0,-17585.0,17678.0,-0.17,-17.45,21.25,0.09,243.0,-0.99,0.64,9.30,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,20.58,116991.0,-25488.0,45006.0,1.26,-17.89,38.47,13.70,163.0,-0.57,0.90,38.08,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.57,H-SC,3.11,114885.0,-28918.0,55662.0,-0.09,-20.11,48.45,18.60,148.0,-0.52,0.88,26.78,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-68.70,H-SC,2.69,215298.0,-54369.0,90705.0,1.79,-20.16,42.13,13.47,142.0,-0.60,1.65,8.89,XY24,NTT,PAINTS
43,INDIGOPNTS,1407.73,1408.0,109.82,M-SC,3.39,137628.0,-36931.0,36967.0,2.88,-21.16,26.86,0.02,221.0,-1.00,1.05,19.34,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.04,H-LC,15.66,238430.0,-57504.0,151403.0,0.53,-19.43,63.50,31.73,2.0,-0.38,1.83,1.79,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.75,H-LC,10.17,256435.0,-5166.0,150784.0,0.31,-1.97,58.80,55.67,4.0,-0.03,1.97,4.01,X40,BTT,IT
89,VBL,492.64,671.64,-9.35,H-LC,10.61,275375.0,10335.0,85972.0,0.22,3.90,31.22,36.33,5.0,0.12,2.11,17.72,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,9.99,197145.0,-19374.0,112274.0,0.16,-8.95,56.95,42.91,7.0,-0.17,1.51,3.25,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.37,H-LC,3.78,128642.0,-136784.0,193696.0,-0.50,-51.53,150.57,21.44,8.0,-0.71,0.99,3.81,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.48,H-MC,13.95,160590.0,34390.0,11787.0,-0.22,27.25,7.34,36.59,111.0,2.92,1.23,43.18,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.48,H-MC,13.95,160590.0,34390.0,11787.0,-0.22,27.25,7.34,36.59,111.0,2.92,1.23,43.18,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.89,130612.0,5315.0,53695.0,-1.36,4.24,41.11,47.09,141.0,0.10,1.00,63.47,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,0.44,H-MC,7.45,173390.0,8720.0,59247.0,-1.59,5.30,34.17,41.28,91.0,0.15,1.33,13.70,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-6.49,H-SC,3.06,106249.0,4245.0,39684.0,-0.09,4.16,37.35,43.06,123.0,0.11,0.81,21.99,X40N,ATH,MISC
68,SBIN,760.30,863.00,-11.34,M-LC,3.93,218426.0,16946.0,10266.0,0.10,8.41,4.70,13.51,61.0,1.65,1.67,22.07,XY25,NTT,BANKS


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-12.51,H-MC,6.88,162638.0,11725.0,45587.0,1.85,7.77,28.03,37.97,92.0,0.26,1.25,35.04,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.09,H-SC,12.56,136671.0,13335.0,120421.0,1.50,10.81,88.11,108.45,119.0,0.11,1.05,35.58,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.89,130612.0,5315.0,53695.0,-1.36,4.24,41.11,47.09,141.0,0.10,1.00,63.47,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.33,M-SC,4.10,94148.0,2650.0,76100.0,-0.53,2.90,80.83,86.07,219.0,0.03,0.72,36.19,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-14.25,M-LC,6.84,150598.0,-347.0,109801.0,0.53,-0.23,72.91,72.51,54.0,-0.00,1.15,5.78,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.37,70080.0,-43469.0,83521.0,0.14,-38.28,119.18,35.27,268.0,-0.52,0.54,71.52,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.83,H-SC,16.83,89457.0,-11506.0,106033.0,-0.06,-11.40,118.53,93.63,149.0,-0.11,0.69,33.24,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,15.63,90765.0,-7215.0,29081.0,0.53,-7.36,32.04,22.32,152.0,-0.25,0.70,31.64,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,85.53,H-SC,20.58,116991.0,-25488.0,45006.0,1.26,-17.89,38.47,13.70,163.0,-0.57,0.90,38.08,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.89,130612.0,5315.0,53695.0,-1.36,4.24,41.11,47.09,141.0,0.10,1.00,63.47,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,946.25,M-SC,7.36,142238.0,-38905.0,84376.0,1.20,-21.48,59.32,25.10,232.0,-0.46,1.09,13.99,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-26.40,H-SC,9.12,219715.0,26695.0,21334.0,0.38,13.83,9.71,24.89,134.0,1.25,1.68,40.41,X40N,NTT,HEALTHCARE
89,VBL,492.64,671.64,-9.35,H-LC,10.61,275375.0,10335.0,85972.0,0.22,3.90,31.22,36.33,5.0,0.12,2.11,17.72,X40N,ATH,FMCG
35,HEROMOTOCO,4311.81,5949.07,-12.51,H-MC,6.88,162638.0,11725.0,45587.0,1.85,7.77,28.03,37.97,92.0,0.26,1.25,35.04,AR,ATH,AUTO
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.37,70080.0,-43469.0,83521.0,0.14,-38.28,119.18,35.27,268.0,-0.52,0.54,71.52,XR,NTT,HOTELS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.42
1,25,41.11
2,50,70.86


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.57
LC    34.10
MC    26.32
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.06
X40N     15.32
X40      13.38
XY25     10.69
XR        8.99
AR        8.43
OX40N     7.58
X5K       2.23
MH        1.77
X200      1.41
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.77
H-SC    24.60
H-MC    23.35
M-SC    13.50
M-LC     7.24
M-MC     2.62
L-SC     1.47
L-LC     1.09
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-10.83,46.46
IT,11.99,-19.31,78.89
FINANCE,8.38,-19.24,63.36
BANKS,7.69,-13.64,61.89
MISC,5.93,-18.76,86.83
PAINTS,5.68,-17.69,40.88
HEALTHCARE,5.23,3.82,20.47
ELECTRICAL,4.69,-14.81,64.53
AUTO,4.44,-16.80,67.62


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2843560.0,23
XR,1120062.0,13
AR,1110344.0,10
X40,826819.0,10
XY25,654424.0,8
X40N,608587.0,12
OX40N,469174.0,10
SR,196256.0,2
X5K,147789.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2807207.0,28
M-SC,1483441.0,17
H-LC,1376766.0,18
H-MC,1332850.0,18
M-LC,432470.0,5
M-MC,334708.0,2
L-SC,268900.0,3
L-MC,57546.0,1
L-LC,44225.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      932912.0      6
M-SC       XY24      868077.0      7
H-SC       AR        670523.0      5
           XR        530870.0      6
H-LC       X40       516813.0      5
H-MC       XY24      488924.0      5
M-MC       XY24      334708.0      2
H-LC       XY25      256827.0      3
           X40N      239253.0      5
           AR        233355.0      2
H-MC       X40       232364.0      4
H-SC       X40N      222707.0      4
H-MC       XY25      222090.0      2
H-SC       OX40N     200244.0      4
           SR        196256.0      2
L-SC       XR        178952.0      2
M-LC       XY24      178908.0      2
M-SC       AR        160879.0      2
H-MC       X40N      146627.0      3
M-LC       X5K       133495.0      1
M-SC       OX40N     130059.0      4
           XR        125768.0      2
           XY25      121016.0      1
H-MC       XR        117125.0      1
M-LC       XR        109801.0      1
L-SC       OX40N      89948.0      1
M-SC       X40        77642.0      1
H-LC       X200       76193.0      1
L-MC       XR         57546.0      1
H-SC       MH         53695.0      1
H-MC       OX40N      48923.0      1
           AR         45587.0      1
L-LC       XY25       44225.0      1
H-LC       XY24       40031.0      1
H-MC       MH         31210.0      1
H-LC       X5K        14294.0      1
M-LC       XY25       10266.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
